In [1]:

### File Address: https://www.sample-videos.com/
%load_ext autoreload
%autoreload 2
import json
import time
import os
from blake3 import blake3
import warnings
warnings.filterwarnings("ignore")
video_path = "demo-3s-small.mp4"
input_height = 240
input_width = 320
output_height = 120
output_width = 160
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
snark_path = "/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds"
target_pk_file = os.path.join(base_dir, "demo", "test_pk.bin")
target_vk_file = os.path.join(base_dir, "demo", "test_vk.bin")


# Init ELF

In [9]:
start_time = time.time()
os.system(f"cd {snark_path}/program-precompile && RUSTFLAGS='-Awarnings' cargo prove build")
end_time = time.time()
print(f"Time taken to build ELF: {end_time - start_time} seconds")

cargo:warning=rustc +succinct --version: "rustc 1.82.0-dev\n"
cargo:rustc-env=SP1_ELF_freivalds-precompile=/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/elf-compilation/riscv32im-succinct-zkvm-elf/release/freivalds-precompile
Time taken to build ELF: 0.34406399726867676 seconds


[sp1]  warning: unused import: `blake3::hash`
[sp1]   --> program-precompile/src/main.rs:3:5
[sp1]    |
[sp1]  3 | use blake3::hash;
[sp1]    |     ^^^^^^^^^^^^
[sp1]    |
[sp1]    = note: `#[warn(unused_imports)]` on by default
[sp1]  
[sp1]  warning: `freivalds-precompile` (bin "freivalds-precompile") generated 1 warning (run `cargo fix --bin "freivalds-precompile"` to apply 1 suggestion)
[sp1]      Finished `release` profile [optimized] target(s) in 0.08s


# Verifier Setup Pk and Vk

In [10]:
start_time = time.time()
config = {
    "target_pk_file": target_pk_file,
    "target_vk_file": target_vk_file
}
with open(os.path.join(snark_path, "init", "config.json"), "w") as f:
    json.dump(config, f)
os.system(f"cd {snark_path}/init && RUST_LOG=info cargo run --release -- --prove")
end_time = time.time()
print(f"Time taken to generate Pk and Vk: {end_time - start_time} seconds")

 --> init/src/main.rs:1:36
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                    ^^^^^^^^^^^^^^^^^^^^^^^^  ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> init/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> init/src/main.rs:4:5
  |
4 | use clap::Parser;
  |     ^^^^^^^^^^^^

  --> init/src/main.rs:13:9
   |
13 |     let client = ProverClient::from_env();
   |         ^^^^^^ help: if this is intentional, prefix it with an underscore: `_client`
   |
   = note: `#[warn(unused_variables)]` on by default

    Finished `release` profile [optimized] target(s) in 0.17s
     Running `/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/release/init --prove`


2025-05-16T05:47:56.998444Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-05-16T05:47:57.488213Z  INFO vk verification: true
2025-05-16T05:47:59.333550Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-05-16T05:47:59.797765Z  INFO vk verification: true
Successfully Save Pk and Vk
Time taken to generate Pk and Vk: 6.075197219848633 seconds


# Prover

### Decompose images from the video

In [11]:
# Use ffmpeg to decompose
import os
# Create directories if they don't exist
if not os.path.exists("frames"):
    os.makedirs("frames")
if not os.path.exists("decomposed_frames"):
    os.makedirs("decomposed_frames")

os.system(f"ffmpeg -i {video_path} frames/output_%04d.png")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

0

In [14]:
import sys
sys.path.append(os.path.abspath("../"))
from scripts.decompose_image import decompose_image 
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("frames", i), os.path.join("decomposed_frames", i[:-4]))

Saved R channel to decomposed_frames/output_0001_R.txt
Saved G channel to decomposed_frames/output_0001_G.txt
Saved B channel to decomposed_frames/output_0001_B.txt
Saved R channel to decomposed_frames/output_0002_R.txt
Saved G channel to decomposed_frames/output_0002_G.txt
Saved B channel to decomposed_frames/output_0002_B.txt
Saved R channel to decomposed_frames/output_0003_R.txt
Saved G channel to decomposed_frames/output_0003_G.txt
Saved B channel to decomposed_frames/output_0003_B.txt
Saved R channel to decomposed_frames/output_0004_R.txt
Saved G channel to decomposed_frames/output_0004_G.txt
Saved B channel to decomposed_frames/output_0004_B.txt
Saved R channel to decomposed_frames/output_0005_R.txt
Saved G channel to decomposed_frames/output_0005_G.txt
Saved B channel to decomposed_frames/output_0005_B.txt
Saved R channel to decomposed_frames/output_0006_R.txt
Saved G channel to decomposed_frames/output_0006_G.txt
Saved B channel to decomposed_frames/output_0006_B.txt
Saved R ch

In [15]:
### Generate the targeted File 
## Shared knowledge between prover and verifier
if os.path.exists("resized.mp4"):
    os.remove("resized.mp4")
os.system(f"ffmpeg -i {video_path} -vf scale=160:120 resized.mp4")
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

Saved R channel to decomposed_resized_frames/output_0001_R.txt
Saved G channel to decomposed_resized_frames/output_0001_G.txt
Saved B channel to decomposed_resized_frames/output_0001_B.txt
Saved R channel to decomposed_resized_frames/output_0002_R.txt
Saved G channel to decomposed_resized_frames/output_0002_G.txt
Saved B channel to decomposed_resized_frames/output_0002_B.txt
Saved R channel to decomposed_resized_frames/output_0003_R.txt
Saved G channel to decomposed_resized_frames/output_0003_G.txt
Saved B channel to decomposed_resized_frames/output_0003_B.txt
Saved R channel to decomposed_resized_frames/output_0004_R.txt
Saved G channel to decomposed_resized_frames/output_0004_G.txt
Saved B channel to decomposed_resized_frames/output_0004_B.txt
Saved R channel to decomposed_resized_frames/output_0005_R.txt
Saved G channel to decomposed_resized_frames/output_0005_G.txt
Saved B channel to decomposed_resized_frames/output_0005_B.txt
Saved R channel to decomposed_resized_frames/output_000

In [16]:
# Generate Proof from rust 
if not os.path.exists("proof"):
    os.makedirs("proof")
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
test_counts = 5
for i in sorted(os.listdir("decomposed_frames")):
    start_time = time.time()
    print("generating proof for ", i)
    if i.endswith(".txt"):
        config = {
            "input_width": input_width,
            "input_height": input_height,
            "output_width": output_width,
            "output_height": output_height,
            "input_file": os.path.join(base_dir, "demo/decomposed_frames", i),
            "target_file": os.path.join(base_dir, "demo/decomposed_resized_frames", i),
            "target_prove_file": os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin"),
            "target_pk_file": target_pk_file
        }
        with open(os.path.join(snark_path, "script", "config.json"), "w") as f:
            json.dump(config, f)
        os.system(f"cd {snark_path}/script && RUST_LOG=info cargo run --release -- --prove")
    end_time = time.time()
    print(f"Time taken to generate proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break    


generating proof for  output_0001_B.txt


   Compiling getrandom v0.2.15
   Compiling blake3 v1.5.5
   Compiling script v0.1.0 (/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/script)
   Compiling rand_core v0.6.4
   Compiling ring v0.17.8
   Compiling rand_chacha v0.3.1
   Compiling ff v0.13.0
   Compiling crypto-bigint v0.5.5
   Compiling signature v2.2.0
   Compiling ff v0.12.1
   Compiling group v0.12.1
   Compiling group v0.13.0
   Compiling rand v0.8.5
   Compiling pairing v0.22.0
   Compiling bls12_381 v0.7.1
   Compiling p3-field v0.2.0-succinct
   Compiling pasta_curves v0.4.1
   Compiling ark-std v0.4.0
   Compiling pasta_curves v0.5.1
   Compiling tower v0.4.13
   Compiling backoff v0.4.0
   Compiling lib v0.1.0 (/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/lib)
   Compiling ark-serialize v0.4.2
   Compiling rustls-webpki v0.102.8
   Compiling p3-matrix v0.2.0-succinct
   Compiling p3-symmetric v0.2.0-succinct
   Compiling ark-ff v0.4.2
   Compiling elliptic-curve v0.13.8
   Compilin

input_path: "/Users/ultraspicy/Developer/fibonacci/demo/decomposed_frames/output_0001_B.txt"
input_path: "/Users/ultraspicy/Developer/fibonacci/demo/decomposed_resized_frames/output_0001_B.txt"
2025-05-16T05:49:45.307977Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-05-16T05:49:45.807719Z  INFO vk verification: true
2025-05-16T05:49:48.023868Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-05-16T05:49:48.023990Z  INFO prove_core: ┌╴setup
2025-05-16T05:49:48.040588Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-05-16T05:49:48.040664Z  INFO prove_core: ┌╴setup
2025-05-16T05:49:48.058029Z  INFO prove_core: │ ┌╴commit
2025-05-16T05:49:48.058048Z  INFO prove_core: │ └╴380 cycles
2025-05-16T05:49:48.095625Z  INFO prove_core: deferred 0 records    
2025-05-16T05:49:48.143627Z  INFO prove_core: Shard Lifted: Index=1, Cluster=59
2025-05-16T05:49:48.143637Z  INFO prove_core: Chip Cpu: 20  -> 20 
2025-05-16T05:49:48.143639Z  INFO prove_core: Chip AddSub: 20  ->

stderr: cycle-tracker-start: main
stderr: cycle-tracker-start: main
stderr: cycle-tracker-end: main


2025-05-16T05:49:48.348138Z  INFO prove_core:generate main traces: close time.busy=204ms time.idle=1.29µs index=0
2025-05-16T05:49:48.407216Z  INFO prove_core: deferred 0 records    
2025-05-16T05:49:48.484560Z  INFO prove_core: Shard Lifted: Index=2, Cluster=59
2025-05-16T05:49:48.484570Z  INFO prove_core: Chip Cpu: 20  -> 20 
2025-05-16T05:49:48.484572Z  INFO prove_core: Chip AddSub: 20  -> 20 
2025-05-16T05:49:48.484573Z  INFO prove_core: Chip Bitwise: 17  -> 17 
2025-05-16T05:49:48.484573Z  INFO prove_core: Chip Mul: 16  -> 16 
2025-05-16T05:49:48.484574Z  INFO prove_core: Chip ShiftRight: 16  -> 16 
2025-05-16T05:49:48.484575Z  INFO prove_core: Chip ShiftLeft: 16  -> 16 
2025-05-16T05:49:48.484576Z  INFO prove_core: Chip Lt: 19  -> 19 
2025-05-16T05:49:48.484581Z  INFO prove_core: Chip MemoryLocal: 14  -> 17 
2025-05-16T05:49:48.484582Z  INFO prove_core: Chip MemoryInstrs: 19  -> 19 
2025-05-16T05:49:48.484583Z  INFO prove_core: Chip Auipc: 8   -> 17 
2025-05-16T05:49:48.484583Z  

...

input_path: "/Users/ultraspicy/Developer/fibonacci/demo/decomposed_frames/output_0002_G.txt"
input_path: "/Users/ultraspicy/Developer/fibonacci/demo/decomposed_resized_frames/output_0002_G.txt"
2025-05-16T05:52:18.038117Z  WARN SP1_PROVER environment variable not set, defaulting to 'cpu'    
2025-05-16T05:52:18.541357Z  INFO vk verification: true
2025-05-16T05:52:20.799843Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-05-16T05:52:20.799933Z  INFO prove_core: ┌╴setup
2025-05-16T05:52:20.816351Z  INFO prove_core: clk = 0 pc = 0x20209c    
2025-05-16T05:52:20.816410Z  INFO prove_core: ┌╴setup
2025-05-16T05:52:20.833059Z  INFO prove_core: │ ┌╴commit
2025-05-16T05:52:20.833077Z  INFO prove_core: │ └╴380 cycles
2025-05-16T05:52:20.870028Z  INFO prove_core: deferred 0 records    
2025-05-16T05:52:20.920541Z  INFO prove_core: Shard Lifted: Index=1, Cluster=59
2025-05-16T05:52:20.920550Z  INFO prove_core: Chip Cpu: 20  -> 20 
2025-05-16T05:52:20.920552Z  INFO prove_core: Chip AddSub: 20  ->

stderr: cycle-tracker-start: main
stderr: cycle-tracker-start: main
stderr: cycle-tracker-end: main


2025-05-16T05:52:21.123719Z  INFO prove_core:generate main traces: close time.busy=203ms time.idle=1.25µs index=0
2025-05-16T05:52:21.180667Z  INFO prove_core: deferred 0 records    
2025-05-16T05:52:21.255358Z  INFO prove_core: Shard Lifted: Index=2, Cluster=59
2025-05-16T05:52:21.255369Z  INFO prove_core: Chip Cpu: 20  -> 20 
2025-05-16T05:52:21.255371Z  INFO prove_core: Chip AddSub: 20  -> 20 
2025-05-16T05:52:21.255372Z  INFO prove_core: Chip Bitwise: 17  -> 17 
2025-05-16T05:52:21.255373Z  INFO prove_core: Chip Mul: 16  -> 16 
2025-05-16T05:52:21.255379Z  INFO prove_core: Chip ShiftRight: 16  -> 16 
2025-05-16T05:52:21.255380Z  INFO prove_core: Chip ShiftLeft: 16  -> 16 
2025-05-16T05:52:21.255381Z  INFO prove_core: Chip Lt: 19  -> 19 
2025-05-16T05:52:21.255382Z  INFO prove_core: Chip MemoryLocal: 14  -> 17 
2025-05-16T05:52:21.255383Z  INFO prove_core: Chip MemoryInstrs: 19  -> 19 
2025-05-16T05:52:21.255384Z  INFO prove_core: Chip Auipc: 8   -> 17 
2025-05-16T05:52:21.255385Z  

stderr: cycle-tracker-end: main


2025-05-16T05:52:21.790449Z  INFO prove_core:generate main traces: close time.busy=310ms time.idle=1.29µs index=2
2025-05-16T05:52:53.853843Z  INFO prove_core: execution report (totals): total_cycles=2346541, total_syscall_cycles=71, touched_memory_addresses=87631
2025-05-16T05:52:53.853854Z  INFO prove_core: execution report (opcode counts):
2025-05-16T05:52:53.853859Z  INFO prove_core:   765058 add
2025-05-16T05:52:53.853865Z  INFO prove_core:   295504 lw
2025-05-16T05:52:53.853866Z  INFO prove_core:   197146 lbu
2025-05-16T05:52:53.853867Z  INFO prove_core:   193067 sw
2025-05-16T05:52:53.853868Z  INFO prove_core:   147713 sll
2025-05-16T05:52:53.853869Z  INFO prove_core:   147492 or
2025-05-16T05:52:53.853870Z  INFO prove_core:   116933 bne
2025-05-16T05:52:53.853871Z  INFO prove_core:   115577 bgeu
2025-05-16T05:52:53.853872Z  INFO prove_core:    96393 mul
2025-05-16T05:52:53.853873Z  INFO prove_core:    92162 srl
2025-05-16T05:52:53.853873Z  INFO prove_core:    74533 xor
2025-05-

# Verifier

In [17]:
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

Saved R channel to decomposed_resized_frames/output_0001_R.txt
Saved G channel to decomposed_resized_frames/output_0001_G.txt
Saved B channel to decomposed_resized_frames/output_0001_B.txt
Saved R channel to decomposed_resized_frames/output_0002_R.txt
Saved G channel to decomposed_resized_frames/output_0002_G.txt
Saved B channel to decomposed_resized_frames/output_0002_B.txt
Saved R channel to decomposed_resized_frames/output_0003_R.txt
Saved G channel to decomposed_resized_frames/output_0003_G.txt
Saved B channel to decomposed_resized_frames/output_0003_B.txt
Saved R channel to decomposed_resized_frames/output_0004_R.txt
Saved G channel to decomposed_resized_frames/output_0004_G.txt
Saved B channel to decomposed_resized_frames/output_0004_B.txt
Saved R channel to decomposed_resized_frames/output_0005_R.txt
Saved G channel to decomposed_resized_frames/output_0005_G.txt
Saved B channel to decomposed_resized_frames/output_0005_B.txt
Saved R channel to decomposed_resized_frames/output_000

In [18]:
test_counts = 3
for i in sorted(os.listdir("decomposed_resized_frames")):
    print("verifying proof for ", i)
    if i.endswith(".txt"):
        start_time = time.time()
        prove_file = os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin")
        config = {
            "target_prove_file": prove_file,
            "target_vk_file": target_vk_file
        }
        with open(os.path.join(snark_path, "verifier", "config.json"), "w") as f:
            json.dump(config, f)
        os.system(f"cd {snark_path}/verifier && cargo run")
        data = []
        with open(os.path.join(base_dir, "demo/decomposed_resized_frames", i), "r") as f:
            for line in f:
                # Split line by spaces and convert each number
                numbers = [int(x) for x in line.strip().split()]
                data.extend(numbers)
        hash_obj = blake3(bytes(data))
        print(hash_obj.hexdigest())
        end_time = time.time()
        print(f"Time taken to verify proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break

verifying proof for  output_0001_B.txt


    Blocking waiting for file lock on package cache
    Blocking waiting for file lock on package cache
   Compiling proc-macro2 v1.0.93
   Compiling unicode-ident v1.0.13
   Compiling cfg-if v1.0.0
   Compiling libc v0.2.158
   Compiling autocfg v1.3.0
   Compiling once_cell v1.19.0
   Compiling serde v1.0.210
   Compiling version_check v0.9.5
   Compiling pin-project-lite v0.2.14
   Compiling byteorder v1.5.0
   Compiling either v1.13.0
   Compiling allocator-api2 v0.2.18
   Compiling syn v1.0.109
   Compiling itoa v1.0.11
   Compiling equivalent v1.0.1
   Compiling subtle v2.6.1
   Compiling crossbeam-utils v0.8.20
   Compiling memchr v2.7.4
   Compiling rayon-core v1.12.1
   Compiling log v0.4.22
   Compiling ahash v0.8.11
   Compiling typenum v1.17.0
   Compiling num-traits v0.2.19
   Compiling tracing-core v0.1.32
   Compiling itertools v0.12.1
   Compiling generic-array v0.14.7
   Compiling libm v0.2.8
   Compiling const-oid v0.9.6
   Compiling radium v0.7.0
   Compiling tap v1.

equal_sum: true
exceed_limit_20: 613
exceed_limit_50: 49
hash_target_image: Hash("c46149be2e66bc0b4899d725e14d3281df75453e961daf6c64fcf19911864990")
c46149be2e66bc0b4899d725e14d3281df75453e961daf6c64fcf19911864990
Time taken to verify proof for output_0001_B.txt: 56.746394872665405 seconds
verifying proof for  output_0001_G.txt


    Blocking waiting for file lock on package cache
 --> verifier/src/main.rs:1:62
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                              ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> verifier/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> verifier/src/main.rs:3:5
  |
3 | use blake3::hash;
  |     ^^^^^^^^^^^^

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 1.35s
     Running `/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/debug/verifier`


equal_sum: true
exceed_limit_20: 121
exceed_limit_50: 0
hash_target_image: Hash("236150d752ae55abe27b9f8407379836f19e2e02cc9478efad79201ac140ffc5")
236150d752ae55abe27b9f8407379836f19e2e02cc9478efad79201ac140ffc5
Time taken to verify proof for output_0001_G.txt: 31.035825967788696 seconds
verifying proof for  output_0001_R.txt


 --> verifier/src/main.rs:1:62
  |
1 | use sp1_sdk::{utils, ProverClient, SP1ProofWithPublicValues, SP1Stdin, SP1VerifyingKey};
  |                                                              ^^^^^^^^  ^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> verifier/src/main.rs:2:5
  |
2 | use std::io::Write;
  |     ^^^^^^^^^^^^^^

 --> verifier/src/main.rs:3:5
  |
3 | use blake3::hash;
  |     ^^^^^^^^^^^^

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 0.25s
     Running `/Users/ultraspicy/Developer/fibonacci/video-resizing-freivalds/target/debug/verifier`


equal_sum: true
exceed_limit_20: 157
exceed_limit_50: 0
hash_target_image: Hash("3129a6601880af99970a4ac518d4ed7f2ee9ae609be67f1e6a67a044be0cd358")
3129a6601880af99970a4ac518d4ed7f2ee9ae609be67f1e6a67a044be0cd358
Time taken to verify proof for output_0001_R.txt: 27.36140489578247 seconds
